In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None
import random
np.random.seed(1991)
from multiprocessing import Pool
seed_list=list(range(1,1000,100))
import numba

In [2]:
@numba.jit(nopython=True)
def year_binner(year,val=10):
    return year - year%val


@numba.jit(nopython=True)
def decremover(dc):

    new_constituents=[]
    for constituent,decades in dc.items():
        if len(decades)==1:
            if 2000 in decades:
                new_constituents.append(constituent)
    return(new_constituents)

In [3]:
modifiers_list=pkl.load( open("modifier_list_reduced.pkl",'rb'))
heads_list=pkl.load( open("head_list_reduced.pkl",'rb'))

In [4]:
compounds=pd.read_pickle("/data/dharp/compounding/datasets/compounds.pkl")
compounds.reset_index(inplace=True)
compounds.year=compounds.year.astype("int32")
compounds=compounds.query('1800 <= year <= 2010').copy()
compounds['time']=year_binner(compounds['year'].values,10)
#compounds = dd.from_pandas(compounds, npartitions=100)
compounds=compounds.groupby(['modifier','head','time'])['count'].sum().to_frame()
compounds.reset_index(inplace=True)
compounds=compounds.loc[compounds.groupby(['modifier','head','time'])['count'].transform('sum').gt(50)]
compounds=compounds.loc[compounds['modifier'].isin(modifiers_list) & compounds['head'].isin(heads_list)]
compounds.info()
compounds.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2713711 entries, 12 to 8892563
Data columns (total 4 columns):
modifier    object
head        object
time        int64
count       float64
dtypes: float64(1), int64(1), object(2)
memory usage: 103.5+ MB


,modifier,head,time,count
12,a_noun,addison_noun,1970,96.000
13,a_noun,addison_noun,1980,206.000
14,a_noun,addison_noun,1990,196.000
15,a_noun,addison_noun,2000,222.000
20,a_noun,anti_noun,1990,67.000


In [5]:
novel_compounds=compounds.loc[compounds.groupby(['modifier','head'])['time'].transform('count').eq(1)]
novel_compounds_2000=novel_compounds.loc[novel_compounds.time==2000]
novel_compounds_2000_list=list(zip(novel_compounds_2000['modifier'].tolist(),novel_compounds_2000['head'].tolist()))
display(len(novel_compounds_2000_list))
novel_compounds_2000_list[:10]

111551

[('a_noun', 'bachelor_noun'),
 ('a_noun', 'bug_noun'),
 ('a_noun', 'christian_noun'),
 ('a_noun', 'corner_noun'),
 ('a_noun', 'cultural_noun'),
 ('a_noun', 'dog_noun'),
 ('a_noun', 'family_noun'),
 ('a_noun', 'force_noun'),
 ('a_noun', 'go_noun'),
 ('a_noun', 'gospel_noun')]

In [6]:
maske_list=novel_compounds.loc[novel_compounds.time==1990]
maske_list=list(zip(maske_list['modifier'].tolist(),maske_list['head'].tolist()))
display(len(maske_list))
maske_list[110:130]

42337

[('abstract_noun', 'unlimited_noun'),
 ('abstract_noun', 'voronoi_noun'),
 ('abu_noun', 'hilal_noun'),
 ('abu_noun', 'salabikh_noun'),
 ('abuja_noun', 'high_noun'),
 ('abulaziz_noun', 'city_noun'),
 ('abundance_noun', 'datum_noun'),
 ('abundance_noun', 'increase_noun'),
 ('abundance_noun', 'level_noun'),
 ('abundance_noun', 'result_noun'),
 ('abundance_noun', 'zone_noun'),
 ('abuse_noun', 'abstract_noun'),
 ('abuse_noun', 'antecedent_noun'),
 ('abuse_noun', 'aspect_noun'),
 ('abuse_noun', 'war_noun'),
 ('abyssinia_noun', 'baptist_noun'),
 ('ac_noun', 'approach_noun'),
 ('ac_noun', 'concept_noun'),
 ('ac_noun', 'current_noun'),
 ('ac_noun', 'published_noun')]

In [7]:
novel_compounds_new=compounds.loc[compounds.groupby(['modifier','head'])['time'].transform('count').eq(2)]
novel_compounds_1990=novel_compounds_new.loc[novel_compounds_new.time.isin([1990,2000])]
novel_compounds_1990_list=list(zip(novel_compounds_1990['modifier'].tolist(),novel_compounds_1990['head'].tolist()))
novel_compounds_1990_list=list(set(novel_compounds_1990_list))
display(len(novel_compounds_1990_list))
novel_compounds_1990_list[:10]

143058

[('distribution_noun', 'p_noun'),
 ('cardiac_noun', 'contractile_noun'),
 ('drawing_noun', 'unit_noun'),
 ('white_noun', 'mythology_noun'),
 ('joseph_noun', 'devlin_noun'),
 ('happy_noun', 'bottom_noun'),
 ('club_noun', 'good_noun'),
 ('sandford_noun', 'stile_noun'),
 ('fund_noun', 'b_noun'),
 ('ai_noun', 'tee_noun')]

In [8]:
total_compounds=compounds.query('1800 <= time <= 1989').copy()
total_compounds_list=list(zip(total_compounds['modifier'].tolist(),total_compounds['head'].tolist()))
total_compounds_list=list(set(total_compounds_list))
display(len(total_compounds_list))
total_compounds_list[:10]

479993

[('rulemaking_noun', 'procedure_noun'),
 ('seller_noun', 'increase_noun'),
 ('civilian_noun', 'relief_noun'),
 ('cod_noun', 'colorado_noun'),
 ('richard_noun', 'loveland_noun'),
 ('sandford_noun', 'stile_noun'),
 ('justice_noun', 'joseph_noun'),
 ('advisory_noun', 'center_noun'),
 ('summer_noun', 'home_noun'),
 ('beeper_noun', 'surveillance_noun')]

In [9]:
compounds.loc[(compounds['modifier']=='bank_noun') & (compounds['head']=='expert_noun') ]

,modifier,head,time,count
531967,bank_noun,expert_noun,1900,75.000
531970,bank_noun,expert_noun,2000,190.000


In [10]:
total_compounds_list=list(set(total_compounds_list).difference(novel_compounds_1990_list))
display(len(total_compounds_list))
total_compounds_list[:10]

447507

[('rulemaking_noun', 'procedure_noun'),
 ('seller_noun', 'increase_noun'),
 ('civilian_noun', 'relief_noun'),
 ('cod_noun', 'colorado_noun'),
 ('richard_noun', 'loveland_noun'),
 ('justice_noun', 'joseph_noun'),
 ('advisory_noun', 'center_noun'),
 ('summer_noun', 'home_noun'),
 ('beeper_noun', 'surveillance_noun'),
 ('h_noun', 'loop_noun')]

In [11]:
compound_list=novel_compounds_1990_list+novel_compounds_2000_list+total_compounds_list

In [12]:
len(set(total_compounds_list).intersection(novel_compounds_2000_list))

0

In [13]:
len(set(novel_compounds_2000_list).intersection(novel_compounds_1990_list))

0

In [14]:
len(set(total_compounds_list).intersection(novel_compounds_1990_list))

0

In [15]:
combined_list=[x+' '+y for x,y in novel_compounds_2000_list]

In [16]:
pkl.dump(total_compounds_list,open('compound_trainset.pkl','wb'))

In [17]:
pkl.dump(novel_compounds_1990_list,open('compound_validset.pkl','wb'))

In [18]:
pkl.dump(novel_compounds_2000_list,open('compound_testset.pkl','wb'))

In [19]:
combined_list=np.random.choice(combined_list,20_000,replace=False)

In [20]:
returned_list=[x.split() for x in combined_list]

## Creation of Negative Classes - CorruptHead and CorruptModifier


In [26]:
def corrupt_dataset_creator(choice):
    random.seed(choice)
    corrupt_modifier_list=[]
    corrupt_head_list=[]
    for cur_num,(modifier,head) in enumerate(returned_list):
    # CorruptHead , Modifier is same
        #print(cur_num)

        while True:
            temp_corrupt_head=random.choice(heads_list)
            if (modifier,temp_corrupt_head) not in compound_list:
                corrupt_head_list.append((modifier,temp_corrupt_head))
                break
    # CorruptModifier , Head is same
        while True:
            temp_corrupt_modifier=random.choice(modifiers_list)
            if (temp_corrupt_modifier,head) not in compound_list:
                corrupt_modifier_list.append((temp_corrupt_modifier,head))
                break
    return corrupt_head_list,corrupt_modifier_list

In [ ]:
pool = Pool(10)
test_results=pool.map(corrupt_dataset_creator,seed_list)

In [9]:
for num,(corrupt_heads,corrupt_modifiers) in enumerate(results):
    print(num+1)
    pkl.dump( corrupt_heads, open( '/data/dharp/compounding/datasets/tescorrupt_heads_'+str(num+1)+'.pkl', "wb" ) )
    
    pkl.dump( corrupt_modifiers, open( '/data/dharp/compounding/datasets/corrupt_modifiers_'+str(num+1)+'.pkl', "wb" ) )

1
2
3
4
5
6
7
8
9
10


In [7]:
results[1][1][:20]

[('conference_n', 'peaceful_n'),
 ('constant_n', 'robinson_n'),
 ('attorney_n', 'supply_n'),
 ('extinction_n', 'peaceful_n'),
 ('reel_n', 'standard_n'),
 ('supreme_n', 'award_n'),
 ('switch_n', 'joint_n'),
 ('shoots_n', 'companion_n'),
 ('tuning_n', 'foundation_n'),
 ('familial_n', 'gentleman_n'),
 ('nothing_n', 'boat_n'),
 ('diasporic_n', 'pay_n'),
 ('scots_n', 'wife_n'),
 ('snap_n', 'othello_n'),
 ('snail_n', 'sage_n'),
 ('locality_n', 'robinson_n'),
 ('theme_n', 'evidence_n'),
 ('vermont_n', 'association_n'),
 ('advocacy_n', 'crane_n'),
 ('aol_n', 'chase_n')]

In [8]:
results[1][0][:20]

[('a_n', 'subtests_n'),
 ('a_n', 'ma_n'),
 ('a_n', 'upset_n'),
 ('india_n', 'denise_n'),
 ('india_n', 'governing_n'),
 ('india_n', 'knot_n'),
 ('india_n', 'managerial_n'),
 ('india_n', 'faith_n'),
 ('india_n', 'clause_n'),
 ('india_n', 'afield_n'),
 ('india_n', 'giles_n'),
 ('india_n', 'astrocyte_n'),
 ('india_n', 'eric_n'),
 ('india_n', 'reel_n'),
 ('india_n', 'bone_n'),
 ('the_n', 'will_n'),
 ('the_n', 'kiln_n'),
 ('the_n', 'palestine_n'),
 ('the_n', 'winner_n'),
 ('the_n', 'america_n')]